In [84]:
#load data 

import pandas as pd

trip_data_file = "data/trip_data_4.csv"
trip_fare_file = "data/trip_fare_4.csv"

trips = pd.read_csv(trip_data_file, nrows=50000)
fares = pd.read_csv(trip_fare_file, nrows=50000)

full_tripdata = pd.merge(left=trips,right=fares, how='left', left_on=['medallion',' hack_license',' vendor_id',' pickup_datetime'], right_on=['medallion',' hack_license',' vendor_id',' pickup_datetime'])
full_tripdata = full_tripdata.drop_duplicates()


In [85]:
full_tripdata.columns

Index([u'medallion', u' hack_license', u' vendor_id', u' rate_code',
       u' store_and_fwd_flag', u' pickup_datetime', u' dropoff_datetime',
       u' passenger_count', u' trip_time_in_secs', u' trip_distance',
       u' pickup_longitude', u' pickup_latitude', u' dropoff_longitude',
       u' dropoff_latitude', u' payment_type', u' fare_amount', u' surcharge',
       u' mta_tax', u' tip_amount', u' tolls_amount', u' total_amount'],
      dtype='object')

In [87]:
full_tripdata = full_tripdata.drop(columns=['medallion',' hack_license',' vendor_id',' rate_code',' store_and_fwd_flag',' payment_type'])

In [70]:
full_tripdata.columns

Index([u' pickup_datetime', u' dropoff_datetime', u' passenger_count',
       u' trip_time_in_secs', u' trip_distance', u' pickup_longitude',
       u' pickup_latitude', u' dropoff_longitude', u' dropoff_latitude',
       u' payment_type', u' fare_amount', u' surcharge', u' mta_tax',
       u' tip_amount', u' tolls_amount', u' total_amount'],
      dtype='object')

In [71]:
import sklearn.preprocessing as preprocessing

le = preprocessing.LabelEncoder() 



In [72]:
payment_type = full_tripdata[[' payment_type']]

payment_type = payment_type.apply(le.fit_transform)

In [73]:
payment_type

,payment_type
0,0
1,0
2,0
3,0
4,0
5,0
6,0
7,0
8,0
9,0


In [74]:
enc = preprocessing.OneHotEncoder()
enc.fix(payment_type)

AttributeError: 'OneHotEncoder' object has no attribute 'fix'

# Prepare time

In [88]:
full_tripdata[" pickup_datetime"] = pd.to_datetime(full_tripdata[" pickup_datetime"])
full_tripdata[" dropoff_datetime"] = pd.to_datetime(full_tripdata[" dropoff_datetime"])

full_tripdata["weekday"] = full_tripdata[" pickup_datetime"].dt.dayofweek
full_tripdata["hourofday"] = full_tripdata[" pickup_datetime"].dt.hour

In [89]:
full_tripdata.columns

Index([u' pickup_datetime', u' dropoff_datetime', u' passenger_count',
       u' trip_time_in_secs', u' trip_distance', u' pickup_longitude',
       u' pickup_latitude', u' dropoff_longitude', u' dropoff_latitude',
       u' fare_amount', u' surcharge', u' mta_tax', u' tip_amount',
       u' tolls_amount', u' total_amount', u'weekday', u'hourofday'],
      dtype='object')

# Remove trips without coordinates 

In [90]:
full_tripdata = full_tripdata.loc[(full_tripdata[' pickup_longitude'] != 0) & (full_tripdata[' pickup_latitude'] != 0) & (full_tripdata[' dropoff_latitude'] != 0) & (full_tripdata[' dropoff_latitude'] != 0)]

In [91]:
full_tripdata.dtypes

 pickup_datetime      datetime64[ns]
 dropoff_datetime     datetime64[ns]
 passenger_count               int64
 trip_time_in_secs             int64
 trip_distance               float64
 pickup_longitude            float64
 pickup_latitude             float64
 dropoff_longitude           float64
 dropoff_latitude            float64
 fare_amount                 float64
 surcharge                   float64
 mta_tax                     float64
 tip_amount                  float64
 tolls_amount                float64
 total_amount                float64
weekday                        int64
hourofday                      int64
dtype: object

In [92]:
X = full_tripdata.iloc[:,2:10]
X = X.as_matrix()


y = full_tripdata.iloc[:,12]

In [93]:
from sklearn.feature_selection import VarianceThreshold

#方差选择法，返回值为特征选择后的数据
#参数threshold为方差的阈值
#VarianceThreshold(threshold=3).fit_transform(X)

In [94]:
import numpy as np
from sklearn.svm import SVR
from sklearn.model_selection import cross_val_score

In [95]:
#preprocessing data 
X2 = np.copy(X)
from sklearn.preprocessing import MinMaxScaler

X2 = MinMaxScaler().fit_transform(X2)

# Training

In [39]:

svr_rbf = SVR(kernel='rbf', C=1e3, gamma=0.1)
#svr_rbf.fit(X, y)
scores1 = cross_val_score(svr_rbf, X, y, cv=4,scoring='neg_mean_squared_error',n_jobs=-1)

In [40]:
scores1

array([-4.7743921 , -5.99841249, -5.2613759 , -5.1560234 ])

In [41]:
scores2 = cross_val_score(svr_rbf, X2, y, cv=4,scoring='neg_mean_squared_error',n_jobs=-1)

In [42]:
scores2

array([-2.95233237, -2.7538575 , -2.35086345, -2.44062428])

In [49]:
from sklearn.neighbors import KNeighborsRegressor
model = KNeighborsRegressor()
scores3 = cross_val_score(model, X2, y, cv=4,scoring='neg_mean_squared_error',n_jobs=-1)

In [50]:
scores3

array([-3.12379517, -2.90682761, -2.70580768, -2.81961397])

In [96]:
from sklearn import linear_model
model = linear_model.SGDRegressor()
scores3 = cross_val_score(model, X2, y, cv=4,scoring='neg_mean_squared_error',n_jobs=-1)

In [97]:
scores3

array([-2.55377323, -6.38301355, -4.22899024, -3.4123678 ])

In [ ]:
from sklearn.neural_network import MLPRegressor

model = MLPRegressor(hidden_layer_sizes=(100,30))
scores4 = cross_val_score(model, X2, y, cv=4,scoring='neg_mean_squared_error',n_jobs=-1)

In [ ]:
scores4